# Trouver et appliquer les règles à entropie nulle


## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [1]:
# -*- coding: utf8 -*-
import codecs,glob
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import networkx as nx
import pickle,yaml
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False

### Préparation des matrices de traits

In [2]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [3]:
rep="/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/Longitudinales/"
rep="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/"
fichiers=glob.glob(rep+"*X-Regles.pkl")
samples=[f.rsplit("/",1)[-1].split("Regles")[0] for f in fichiers]
samples={int(s.split("-")[1]):s for s in samples}
samples

{0: 'Longitudinal-00-T10000-F3663-X-',
 1: 'Longitudinal-01-T20000-F5816-X-',
 2: 'Longitudinal-02-T30000-F7497-X-',
 3: 'Longitudinal-03-T40000-F8900-X-',
 4: 'Longitudinal-04-T50000-F10082-X-',
 5: 'Longitudinal-05-T60000-F11252-X-',
 6: 'Longitudinal-06-T70000-F12207-X-',
 7: 'Longitudinal-07-T80000-F13155-X-',
 8: 'Longitudinal-08-T90000-F14038-X-',
 9: 'Longitudinal-09-T100000-F14858-X-',
 10: 'Longitudinal-10-T110000-F15616-X-',
 11: 'Longitudinal-11-T120000-F16280-X-',
 12: 'Longitudinal-12-T130000-F16948-X-',
 13: 'Longitudinal-13-T140000-F17587-X-',
 14: 'Longitudinal-14-T150000-F18240-X-',
 15: 'Longitudinal-15-T160000-F18824-X-',
 16: 'Longitudinal-16-T170000-F19369-X-',
 17: 'Longitudinal-17-T180000-F19921-X-',
 18: 'Longitudinal-18-T190000-F20415-X-',
 19: 'Longitudinal-19-T200000-F20877-X-',
 20: 'Longitudinal-20-T210000-F21393-X-',
 21: 'Longitudinal-21-T220000-F21883-X-',
 22: 'Longitudinal-22-T230000-F22343-X-',
 23: 'Longitudinal-23-T240000-F22786-X-',
 24: 'Longitudi

In [4]:
sample=samples[50]
fRulesPMS="Regles.pkl"
fRulesPMO="Morphomes-Regles.pkl"

# Classe pour la gestion des patrons, des classes et des transformations

In [5]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print "le numéro de forme n'est pas dans [1,2]",n

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme,contextFree=False):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if contextFree:
                filterF1=".*"+patron.split("-")[0]+"$"
            else:
                filterF1=self.patrons[patron]
            if re.match(filterF1,forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if contextFree:
                nbClasse=self.nbClasseCF
                classe=self.classeCF
            else:
                nbClasse=self.nbClasse
                classe=self.classe
            if idClasseForme in nbClasse:
                nTotal=nbClasse[idClasseForme]
                for patron in classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(classe[idClasseForme][patron])/nTotal
            else:
#                if debug: 
#                    print (forme, file=logfile)
#                    print ("pas de classe",idClasseForme, file=logfile)
#                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
#        else:
#            if debug:
#                print (forme, file=logfile) 
#                print ("pas de patron", file=logfile)
        return sortieForme
        

## Ouvrir les fichiers de règles

In [6]:
def openRules(numero,typeRules="PMS"):
    rules=None
    if typeRules=="PMS" and numero in samples:
        with open(rep+samples[numero]+fRulesPMS, 'rb') as input:
            rules = pickle.load(input)
    elif typeRules=="PMO" and numero in samples:
        with open(rep+samples[numero]+fRulesPMO, 'rb') as input:
            rules = pickle.load(input)
    return rules

In [46]:
numero=129
typeRules="PMO"
avecContexte=False

In [47]:
rules=openRules(numero,typeRules=typeRules)

In [70]:
rules[(u'ii1S', u'is1P')].classe

{u'6nE-isj\xf4, E-asj\xf4': {u'6nE-isj\xf4': 1},
 u'6vE-ysj\xf4, E-asj\xf4': {u'6vE-ysj\xf4': 1},
 u'6zE-isj\xf4, E-asj\xf4': {u'6zE-isj\xf4': 1},
 u'E-asj\xf4': {u'E-asj\xf4': 5},
 u'EtE-fysj\xf4, E-asj\xf4': {u'EtE-fysj\xf4': 1},
 u'EtE-isj\xf4, E-asj\xf4': {u'EtE-isj\xf4': 2},
 u'avE-ysj\xf4': {u'avE-ysj\xf4': 1}}

In [51]:
zeroEntropie=[]
for paire in rules:
    classe=rules[paire].classe
    c1,c2=paire
    if c1!=c2:
        zero=True
        print paire
        for transformation in classe:
            choix=classe[transformation]
            if len(choix)>1:
                print choix
                zero=False
        if zero:
            print "zéro",choix
            zeroEntropie.append(paire)
        print

(u'pI2S', u'ps2P')
{u'-tje': 2, u'-je': 1}

(u'ai1S', u'pc2P')
{u'jE-rje': 5, u'E-6rje': 6}
{u'i-Etrije': 5, u'-rje': 1}
{u'i-rije': 1, u'-rje': 1}
{u'i-rije': 11, u'-rje': 5}
{u'y-6vrije': 2, u'y-Orje': 1}

(u'is1P', u'ii2P')
zéro {u'fysj\xf4-Etje': 1}

(u'ai2S', u'fi1P')
{u'y-6vr\xf4': 2, u'y-Or\xf4': 1}
{u'i-r\xf4': 29, u'-r\xf4': 36}
{u'-r\xf4': 1, u'y-ir\xf4': 3, u'y-Er\xf4': 2}
{u'a-6r\xf4': 4, u'6.a-E.6r\xf4': 24}
{u'a-6r\xf4': 16, u'ja-r\xf4': 22}
{u'-r\xf4': 1, u'i-6r\xf4': 2}
{u'i-war\xf4': 1, u'i-jEr\xf4': 1}
{u'-r\xf4': 4, u'i-Etr\xf4': 10}
{u'i-r\xf4': 2, u'i-Err\xf4': 2}
{u'y-Er\xf4': 1, u'y-6vr\xf4': 1}
{u'-r\xf4': 2, u'i-6r\xf4': 5}
{u'-r\xf4': 5, u'i-r\xf4': 3, u'i-Er\xf4': 1}

(u'ai2S', u'fi2P')
{u'-re': 1, u'y-Ere': 1, u'y-ire': 3}
{u'i-re': 23, u'-re': 20}
{u'a-6re': 4, u'6.a-E.6re': 18}
{u'i-Etre': 8, u'-re': 3}
{u'-re': 1, u'i-6re': 2}
{u'a-6re': 16, u'ja-re': 18}
{u'i-re': 2, u'i-Ere': 1, u'-re': 1}
{u'y-6vre': 4, u'y-Ore': 1}
{u'y-Ere': 1, u'y-6vre': 1}

(u'ppFP

{u'-6rE': 1, u'-mirE': 3}
{u'-rE': 8, u'-vrE': 4}
{u'-rE': 2, u'wa-ErE': 4}
{u'-drE': 24, u'-tirE': 3}
{u'-irE': 1, u'-6rE': 2}
{u'-drE': 4, u'-rE': 184}
{u'-rE': 7, u'-j6rE': 3}
{u'j-rE': 2, u'-6rE': 8}
{u'-rE': 1, u'wa-jErE': 1}
{u'-drE': 4, u'-6rE': 50, u'-tirE': 2}
{u'-irE': 8, u'-6rE': 12}
{u'e-ErE': 1, u'je-warE': 1}
{u'E-airE': 1, u'E-OrE': 1}
{u'-trE': 3, u'-rE': 2, u'-j6rE': 1}
{u'j-rE': 3, u'-6rE': 2}
{u'wa-6vrE': 6, u'-rE': 2}
{u'-6rE': 14, u'-tirE': 2}
{u'-6rE': 4, u'-virE': 2}
{u'-rE': 3, u'-6rE': 58}

(u'ppFP', u'is2P')
zéro {u'-sje': 2}

(u'inf', u'pi3P')
{u'r-z': 7, u'r-s': 64, u'ir-': 16}
{u'6..e-2..': 1, u'6..e-9..': 1}
{u'r-z': 21, u'r-s': 19, u'r-': 2}
{u'r-s': 32, u'ir-': 19, u'r-': 1}
{u'r-s': 4, u'r-v': 6}
{u'e-': 70, u'je-': 63}
{u'6.e-2.': 4, u'6.e-9.': 1, u'6.e-E.': 12}
{u'e-': 1, u'ale-v\xf4': 1}
{u'e-': 296, u'O.e-o.': 71}
{u'r-z': 1, u'r-s': 10}
{u'r-': 18, u'tr-s': 17}
{u'r-s': 23, u'r-': 1}
{u'r-s': 4, u'ir-': 4}
{u'6.e-2.': 1, u'6.e-E.': 1}
{u'r-': 2, u'

{u'rje-': 2, u'rje-j': 2}
{u'6rje-': 1, u'O.6rje-o.': 6}
{u'6rje-': 1, u'6rje-E': 1}
{u'j6rje-': 2, u'6rje-': 3}
{u'rje-': 2, u'tirje-': 6}

(u'ai1P', u'ai2S')
zéro {u'm-': 349}

(u'ii1S', u'fi1P')
{u'6.E-E.6r\xf4': 7, u'E-6r\xf4': 2}
{u'avE-Or\xf4': 1, u'E-6r\xf4': 1}
{u'sE-r\xf4': 5, u'E-6r\xf4': 2}
{u'jE-r\xf4': 21, u'E-6r\xf4': 17}
{u'6.E-E.6r\xf4': 7, u'E-r\xf4': 3, u'E-6r\xf4': 1}
{u'zE-r\xf4': 14, u'E-6r\xf4': 9}
{u'E-6r\xf4': 7, u'E-r\xf4': 3, u'vE-r\xf4': 3}
{u'6.E-E.6r\xf4': 10, u'E-6r\xf4': 1}
{u'alE-Odr\xf4': 2, u'E-6r\xf4': 3}
{u'E-r\xf4': 53, u'E-ir\xf4': 12, u'E-6r\xf4': 388}
{u'sE-r\xf4': 76, u'E-6r\xf4': 2}
{u'zE-r\xf4': 5, u'E-6r\xf4': 10}
{u'wajE-Er\xf4': 4, u'jE-r\xf4': 1}
{u'sE-tr\xf4': 11, u'E-6r\xf4': 32}
{u'jE-r\xf4': 1, u'wajE-jEr\xf4': 1}
{u'6.E-E.6r\xf4': 1, u'zE-r\xf4': 4}
{u'avE-Or\xf4': 1, u'E-6r\xf4': 2}
{u'E-ir\xf4': 11, u'E-6r\xf4': 138}
{u'jE-r\xf4': 7, u'E-6r\xf4': 1}
{u'EjE-war\xf4': 1, u'EjE-jEr\xf4': 1}

(u'pi1S', u'pc1P')
{u'E-Orj\xf4': 1, u'-rj\x

{u'y-wa': 12, u'y-E': 5}
{u'iz-je': 2, u'iz-wa': 2}
{'-': 1, u'vi-': 5}
{u'e-': 172, u'O.e-o.': 65}
{u'e-': 70, u'je-': 53}
{u'y-i': 1, '-': 2}
{u'e-E': 1, u'6.e-E.': 13}
{u'e-': 95, u'O.e-o.': 2}
{u'e-': 3, u'te-pt': 2}
{u'6.e-9.': 5, u'6.e-E.': 54}
{u'y-wa': 2, u'y-E': 1}

(u'is2P', u'pi1S')
{u'asje-': 1, u'jasje-': 1}

(u'ps1S', u'is1S')
{u'-as': 32, u'-is': 29}
{u'E.-6.as': 2, u'-as': 4}

(u'pI2S', u'fi1P')
{u'-r\xf4': 2, u'-j6r\xf4': 1}
{u'-dr\xf4': 22, u'-tir\xf4': 3}
{u'j-r\xf4': 4, u'-6r\xf4': 6}
{u'-r\xf4': 128, u'-dr\xf4': 2}
{u'wa-6vr\xf4': 3, u'-r\xf4': 2}
{u'-6r\xf4': 9, u'-tir\xf4': 2}
{u'-6r\xf4': 2, u'-mir\xf4': 2}
{u'-r\xf4': 1, u'wa-jEr\xf4': 1}
{u'-r\xf4': 6, u'-vr\xf4': 4}
{u'E-air\xf4': 1, u'E-Or\xf4': 1}
{u'-6r\xf4': 1, u'-vir\xf4': 1}
{u'-r\xf4': 3, u'-6r\xf4': 41}
{u'e-Er\xf4': 1, u'je-war\xf4': 1}
{u'-ir\xf4': 6, u'-6r\xf4': 2}
{u'-dr\xf4': 4, u'-6r\xf4': 44, u'-tir\xf4': 2}
{u'-r\xf4': 6, u'-tr\xf4': 4, u'-j6r\xf4': 3}

(u'ppFP', u'fi1S')
{u'y-rE': 29, u'y-irE


(u'ps1S', u'pi1S')
{u'EJ-\xea': 11, '-': 2}
{'-': 1, u'p-': 3}
{'-': 7, u'v-': 18}
{u'wa-je': 2, '-': 2}
{u'j-': 2, '-': 3}
{'-': 2, u'-j': 2}
{'-': 19, u's-': 117}
{u'z-': 31, '-': 24}
{u'Ej-je': 1, u'Ej-wa': 1}
{u't-': 28, '-': 49}
{u'd-': 27, '-': 4}

(u'ps2P', u'pi1P')
zéro {u'e-\xf4': 1}

(u'pi3P', u'is1S')
{u'-as': 54, u'-is': 30}
{u'i-jas': 8, u'-jas': 2}
{u'E.-6.as': 2, u'-as': 7}

(u'pi3S', u'is2P')
zéro {u'a-ysje': 1}

(u'is1S', u'ii1S')
{u'is-EtE': 2, u'is-E': 1}
{u's-zE': 1, u'is-E': 9}
{u'ys-6vE': 1, u'ys-EzE': 1}
{u'ys-6vE': 1, u'ys-avE': 1}

(u'ii1P', u'ii2P')
zéro {u'\xf4-e': 518}

(u'pi1S', u'ppMP')
{u'-e': 8, u'-vi': 2}
{u'-e': 98, u'E.-6.e': 51}
{u'i-y': 3, u'i-je': 8, '-': 18, u'-je': 7}
{u'-vi': 1, u'sHi-Ete': 1}
{u'-e': 126, u'-dy': 6, u'-ti': 2}
{u'-He': 32, '-': 3}
{u'-je': 1, u'-ty': 1, '-': 5}
{'-': 2, u'-we': 17}
{u'-e': 23, u'-ti': 2}
{u'-ty': 1, u'vE-ale': 1}
{u'i-je': 41, '-': 109, u'-je': 6}
{'-': 5, u'E-y': 8, u'-je': 9}
{u'-je': 2, '-': 29}
{u'i-je': 3

In [58]:
zeroPop={}
for paire in zeroEntropie:
    for t in rules[paire].classe:
        pop=0
        for c in rules[paire].classe[t]:
            pop+=rules[paire].classe[t][c]
        zeroPop[paire]=pop
    

In [62]:
for k in sorted(zeroPop, key=zeroPop.get,reverse=True):
    print k, zeroPop[k]

(u'ppFP', u'ppMP') 3271
(u'pP', u'pI1P') 1058
(u'ai3P', u'ai1S') 883
(u'ai1S', u'ai3P') 883
(u'pi1P', u'ii1P') 645
(u'ii1P', u'ii2P') 518
(u'ai2S', u'ai1P') 349
(u'ai1P', u'ai2S') 349
(u'ai3P', u'ai2S') 278
(u'ai2S', u'ai3P') 278
(u'ai1S', u'ai1P') 227
(u'ai1P', u'ai1S') 227
(u'ai2S', u'is1S') 195
(u'is1S', u'ai2S') 195
(u'ai2S', u'ai1S') 170
(u'ai1S', u'ai2S') 170
(u'ai3P', u'is1S') 96
(u'ai1P', u'ai3P') 84
(u'ai3P', u'ai1P') 84
(u'ai1P', u'is1S') 82
(u'is1S', u'ai1P') 82
(u'is1S', u'ai3P') 71
(u'ai1S', u'is1S') 69
(u'is1S', u'ai1S') 69
(u'pP', u'ii1P') 54
(u'ps2P', u'ps1S') 48
(u'ps2P', u'pi3P') 46
(u'ps2P', u'pI2S') 41
(u'ps2P', u'pi1S') 39
(u'ps2P', u'pi3S') 38
(u'ps2P', u'pi2S') 37
(u'fi1P', u'pc1P') 36
(u'ii1S', u'ii1P') 34
(u'ps1P', u'pP') 30
(u'pP', u'ii2P') 27
(u'pI2P', u'ii1P') 25
(u'pc2P', u'pc1P') 24
(u'pi2P', u'ii1P') 22
(u'ps1P', u'fi1S') 21
(u'ps1P', u'pc3P') 18
(u'ps1P', u'fi2P') 16
(u'ai2P', u'ai2S') 14
(u'ps1P', u'ps2P') 14
(u'ps2P', u'ps1P') 14
(u'ai2S', u'ai2P') 14


In [74]:
fSamplePMS="paradigmes.csv"
fSamplePMO="Morphomes-paradigmes.csv"

In [157]:
def openSample(numero,typeSample="PMS"):
    result=None
    fSample=""
    if typeSample=="PMS" and numero in samples:
        fSample=rep+samples[numero]+fSamplePMS
    elif typeSample=="PMO" and numero in samples:
        fSample=rep+samples[numero]+fSamplePMO
    if fSample:
        result=pd.read_csv(rep+samples[numero]+fSamplePMS,sep=";",index_col=0,keep_default_na=False,encoding="utf8")
    return result    

In [193]:
df=openSample(50,"PMS")

In [194]:
def makeZeros(sampleFormes,inFormes=True):
    series=sampleFormes.copy()
    series={k:v for k,v in series.iteritems() if v!=""}

    paires=rules.keys()
    formesC={}
    for c in series:
        if not c in formesC:
            formesC[c]=set()
        formesC[c].add(series[c])
        if debug: print c, series[c]
        formeC=series[c]
        pairesC=[(c1,c2) for (c1,c2) in paires if c1==c]
        for p in pairesC:
            formeP=rules[p].sortirForme(series[c])
            if formeP and len(formeP)==1:
                p1=p[1]
                if debug: print p1,formeP
                if p1 not in formesC:
                    formesC[p1]=set()
                formesC[p1].add(formeP.keys()[0])
    if inFormes:
        formesZero={k:list(v)[0] for k,v in formesC.iteritems() if len(v)==1}
    else:
        formesZero={k:list(v)[0] for k,v in formesC.iteritems() if len(v)==1 and k not in series}

    return formesZero

In [200]:
cases=[c for c in df.columns.tolist() if c!="lexeme"]
cases=sorted(cases)

In [223]:
lexemes=u"abreuver clore paître distraire être avoir devoir pouvoir".split(" ")

In [225]:
for lexeme in lexemes:
    index=df.loc[df['lexeme']==lexeme].index.values
    if index:
        i=index[0]
        sampleFormes=df.iloc[i].to_dict()
        sampleFormes={k:v for k,v in sampleFormes.iteritems() if v!=""}
        lexemeI=sampleFormes["lexeme"]
        print lexemeI
        formesZero=makeZeros(sampleFormes,inFormes=False)
        for case in cases:
            if case in sampleFormes:
                print "\t",case,"\t  ",sampleFormes[case]
            elif case in formesZero:
                print "\t",case,"\t=>",formesZero[case]
            else:
                print "\t",case
        print

abreuver
	ai1P
	ai1S
	ai2P 	=> abr9vat
	ai2S 	=> abr6va
	ai3P
	ai3S
	fi1P 	=> abr9v6rô
	fi1S 	=> abr6v6rE
	fi2P 	=> abr6v6re
	fi2S 	=> abr6v6ra
	fi3P
	fi3S 	   abr6v6ra
	ii1P
	ii1S 	=> abr6vE
	ii2P 	=> abr6vje
	ii2S
	ii3P
	ii3S
	inf 	   abr6ve
	is1P
	is1S
	is2P
	is2S
	is3P
	is3S
	pI1P 	=> abr6vô
	pI2P 	=> abr6ve
	pI2S 	=> abr9v
	pP 	   abr6vâ
	pc1P 	=> abr9v6rjô
	pc1S
	pc2P
	pc2S
	pc3P 	=> abr6v6rE
	pc3S
	pi1P 	=> abr6vô
	pi1S 	=> abr9v
	pi2P 	=> abr6ve
	pi2S
	pi3P 	   abr9v
	pi3S 	   abr9v
	ppFP 	=> abr6ve
	ppFS
	ppMP 	=> abr6ve
	ppMS 	   abr6ve
	ps1P
	ps1S
	ps2P
	ps2S
	ps3P
	ps3S

clore
	ai1P
	ai1S
	ai2P
	ai2S
	ai3P
	ai3S
	fi1P
	fi1S
	fi2P
	fi2S 	=> klOra
	fi3P
	fi3S
	ii1P
	ii1S
	ii2P
	ii2S
	ii3P
	ii3S
	inf 	   klOr
	is1P
	is1S
	is2P
	is2S
	is3P
	is3S
	pI1P
	pI2P 	=> kloze
	pI2S 	   klo
	pP 	=> klOâ
	pc1P 	=> klorjô
	pc1S
	pc2P 	=> klodrije
	pc2S
	pc3P
	pc3S
	pi1P
	pi1S 	   klo
	pi2P 	=> kloze
	pi2S 	=> klo
	pi3P 	=> kloz
	pi3S 	   klo
	ppFP 	   kloz
	ppFS 	   kloz
	ppMP 	   klo
	ppM

In [178]:
for i in range(500):
    sampleFormes=df.loc[i].to_dict()
    lexemeI=sampleFormes["lexeme"]
    if not lexemeI.endswith("er"):
        print lexemeI
        numero=1
        for k,v in sampleFormes.iteritems():
            if k!="lexeme" and v!="":
                print numero,k,v
                numero+=1
        print
        formesZero=makeZeros(sampleFormes,inFormes=False)
        for k,v in formesZero.iteritems():
            if k!="lexeme":
                print numero,k,v
                numero+=1
        print

abasourdir
1 ppFS abazurdi
2 ppMS abazurdi


abattre
1 ii1P abatjô
2 ii1S abatE
3 pP abatâ
4 ai3S abati
5 ai3P abatir
6 ps3S abat
7 ii3S abatE
8 inf abatr
9 ii3P abatE
10 pi2S aba
11 pi2P abate
12 ppFS abaty
13 ppFP abaty
14 fi2P abatre
15 pi1S aba
16 pi1P abatô
17 pi3P abat
18 pi3S aba
19 ppMS abaty
20 pc3P abatrE
21 fi3S abatra
22 fi3P abatrô
23 ppMP abaty
24 fi1S abatrE

25 ps1P abatjô
26 pc1P abatriô
27 ai2S abati
28 fi1P abatrô
29 pc2P abatrije
30 ii2P abatje
31 ai1P abatim
32 ai1S abati
33 pI2P abate
34 pI2S aba
35 pI1P abatô
36 fi2S abatra
37 ps1S abat

abolir
1 pP abOlisâ
2 ai3S abOli
3 ii3S abOlisE
4 inf abOlir
5 ppFS abOli
6 pi1S abOli
7 pi1P abOlisô
8 ppMS abOli
9 fi3S abOlira

10 ii1P abOlisjô
11 ii1S abOlisE
12 is1S abOlisas
13 ai1P abOlisam
14 ai1S abOly
15 pi2S abOli
16 ps1S abOlis
17 pi2P abOlise
18 pI2S abOli
19 fi2P abOlire
20 fi2S abOlira
21 pI2P abOlise
22 ai2P abOlisat
23 ii2P abOlisje
24 pi3P abOlis
25 pi3S abOli
26 pc3P abOlirE
27 pc1P abOlirjô
28 is1P abOlisasjô

16 is1P afEblisasjô
17 pI1P afEblisô
18 ai3P afEblir
19 ppMP afEbli
20 fi1P afEblirô
21 ai2S afEbli
22 fi1S afEblirE
23 ii2P afEblisje
24 pi1S afEbli
25 pi1P afEblisô
26 pi2P afEblise
27 pI2S afEbli
28 fi2P afEblire
29 fi2S afEblira
30 ps1S afEblis

affermir
1 ii3S afErmisE
2 inf afErmir
3 pi3S afErmi

4 pI1P afErmisô
5 pI2S afErmi
6 ps1P afErmijô
7 pc3P afErmirE
8 fi1S afErmirE
9 pc1P afErmirjô
10 ai3P afErmir
11 fi1P afErmirô
12 pc2P afErmirje
13 ai2S afErmi
14 pi1S afErmi
15 ii2P afErmje
16 ai1P afErmim
17 ai1S afErmi
18 pi2S afErmi
19 fi2P afErmire
20 fi2S afErmira

affranchir
1 pP afrâSisâ
2 ai3S afrâSi
3 ps3S afrâSis
4 inf afrâSir
5 ppFS afrâSi
6 ppFP afrâSi
7 pi1S afrâSi
8 pi3S afrâSi
9 ppMS afrâSi
10 ppMP afrâSi

11 ii1P afrâSisjô
12 ii1S afrâSisE
13 is1S afrâSis
14 ai3P afrâSir
15 ai1P afrâSim
16 pi2S afrâSi
17 ps1S afrâSis
18 pi2P afrâSise
19 pI2S afrâSi
20 fi2P afrâSire
21 fi2S afrâSira
22 pI2P afrâSise
23 ai2S afrâSi
24 pc2P afrâSirje
25 ii2P afrâSisje
26 pi1P afrâSisô
27 p

22 fi2S aplatira
23 pI2P aplatise
24 ai2S aplati
25 pc2P aplatirje
26 ii2P aplatisje
27 pi1P aplatisô
28 pi3P aplatis
29 pc3P aplatirE
30 pc1P aplatirjô
31 fi1P aplatirô
32 fi1S aplatirE
33 pI1P aplatisô

apparaître
1 pP aparEsâ
2 ai3S apary
3 ai3P aparyr
4 ps3S aparEs
5 ii3S aparEsE
6 inf aparEtr
7 ii3P aparEsE
8 pi2P aparEse
9 ppFS apary
10 ppFP apary
11 fi2S aparEtra
12 ii2P aparEsje
13 pi1S aparE
14 pi1P aparEsô
15 pi3P aparEs
16 pi3S aparE
17 ppMS apary
18 pc3S aparEtrE
19 pc3P aparEtrE
20 fi3S aparEtra
21 fi3P aparEtrô
22 ppMP apary
23 fi1P aparEtrô
24 fi1S aparEtrE

25 ii1P aparEsjô
26 ii1S aparEsE
27 pI2S aparE
28 is2P aparysje
29 pc1P aparEtriô
30 pI1P aparEsô
31 is1S aparys
32 ai2S apary
33 ai1P aparym
34 ai1S apary
35 pI2P aparEse
36 pi2S aparE
37 fi2P aparEtre
38 ps1S aparEs

appartenir
1 ii1P apart6njô
2 ii1S apart6nE
3 pP apart6nâ
4 is3S apartê
5 ps3S apartjEn
6 ii3S apart6nE
7 inf apart6nir
8 ii3P apart6nE
9 pi2P apart6ne
10 ii2P apart6nje
11 pi1S apartjê
12 pi1P apart6n

6 pc1P asyZEtirjô
7 ai3P asyZEtir
8 ps2P asyZEtije
9 pI2S asyZEti
10 ps1P asyZEtijô
11 pc3P asyZEtirE
12 fi1S asyZEtirE
13 is1S asyZEtis
14 ai2P asyZEtit
15 ppMP asyZEti
16 fi1P asyZEtirô
17 pc2P asyZEtirje
18 ai2S asyZEti
19 pi1S asyZEti
20 ai1P asyZEtim
21 ai1S asyZEti
22 pi2S asyZEti
23 fi2P asyZEtire
24 fi2S asyZEtira

astreindre
1 ii1P astrEJô
2 ii3S astrEJE
3 inf astrêdr
4 ppFS astrêt
5 pi1S astrê
6 pi3S astrê
7 ppMS astrê

8 pI2P astrEJe
9 ii1S astrEJE
10 ai3P astrEJir
11 ppFP astrêt
12 pI2S astrê
13 ps1P astrêjô
14 pc3P astrêdrE
15 pP astrEJâ
16 pI1P astrEJô
17 ppMP astrê
18 fi1P astrêdrô
19 ai2S astrEJi
20 ii2P astrEJe
21 ai1P astrEJim
22 ai1S astrEJi
23 pi2P astrEJe
24 pi2S astrê
25 pi1P astrEJô
26 pi3P astrEJ
27 ps1S astrEJ

atteindre
1 ii1P atEJô
2 ii1S atEJE
3 pP atEJâ
4 ai3S atEJi
5 ai3P atEJir
6 ps3S atEJ
7 ii3S atEJE
8 ai1P atEJim
9 inf atêdr
10 ii3P atEJE
11 pi2S atê
12 ai1S atEJi
13 ppFS atêt
14 ppFP atêt
15 fi2P atêdre
16 pi1S atê
17 pi1P atEJô
18 pi3P atEJ
19 pi3S a

9 ai1S bufi
10 pi2S bufi
11 inf bufir
12 ps1S bufis
13 pi2P bufise
14 pI2S bufi
15 fi2P bufire
16 fi2S bufira
17 pI2P bufise
18 ai2P bufit
19 ai2S bufi
20 pc2P bufirje
21 ii2P bufisje
22 pi1S bufi
23 pi1P bufisô
24 pi3S bufi
25 pc3P bufirE
26 pc1P bufirjô
27 is1P bufisjô
28 fi1P bufirô
29 fi1S bufirE

bouillir
1 ii1S bujE
2 ii3S bujE
3 inf bujir
4 pi2S bu
5 ppFS buji
6 pi1S bu
7 pi3S bu

8 ii1P bujô
9 pP bujâ
10 ps2P buje
11 pI2P buje
12 pI2S bu
13 pc3P bujirE
14 is1S bujis
15 pI1P bujô
16 ppMP buji
17 fi1P bujirô
18 fi1S bujirE
19 ii2P buje
20 pi3P buj
21 pi1P bujô
22 ps1S buj
23 ps1P bujô
24 fi2P bujire
25 fi2S bujira
26 pi2P buje

brandir
1 ii1S brâdisE
2 pP brâdisâ
3 ai3S brâdi
4 ps3S brâdis
5 ii3S brâdisE
6 inf brâdir
7 ii3P brâdisE
8 ppFS brâdi
9 ppFP brâdi
10 pi3S brâdi
11 ppMS brâdi
12 fi3P brâdirô

13 ii1P brâdisjô
14 is1S brâdis
15 ai3P brâdir
16 ai1P brâdim
17 ai1S brâdi
18 pi2S brâdi
19 pi2P brâdise
20 ps1S brâdis
21 pI2S brâdi
22 fi2P brâdire
23 fi2S brâdira
24 pI2P brâdis

In [236]:
rules[("pi3S","pi1S")].sortirForme("Xi")

{'Xi': 1.0}